In [1]:
def to_categorical(y):
    from keras import backend as k
    return k.one_hot(y, 512)

In [2]:
from keras.layers import Input, Dense, Concatenate,Lambda,Reshape,Dropout
from keras.models import Model
from keras.optimizers import Adam
import numpy as np

vector = Input(shape=(512,), name='Vectors-Input',dtype='float32')
vector_shiter = Dense(512)(vector)
position = Input(shape=(1,), name='Positions-Input',dtype='int32')
categorical_position=Lambda(to_categorical, name='Positional_encoding')(position)
reshaped_categorical_position=Reshape((512,))(categorical_position)
concatenated_encoder_input=Concatenate()([vector_shiter,reshaped_categorical_position])
encoder_input_divider1 = Dense(1024, name='Encoder-Output-Divider-1',activation='selu')(concatenated_encoder_input)
encoder_input_dropout1=Dropout(0.1, name='Encoder-Output-Dropout-1')(encoder_input_divider1)

encoder_input_divider2 = Dense(512, name='Encoder-Output-Divider-2',activation='selu')(encoder_input_dropout1)
encoder_input_dropout2=Dropout(0.1, name='Encoder-Output-Dropout-2')(encoder_input_divider2)

output_vector = Dense(512, name='Encoder-Garbedge',activation='selu')(encoder_input_dropout2)
encoder_input_divider3 = Dense(256, name='Encoder-Output-Divider-3',activation='selu')(encoder_input_dropout2)

output_vector2 = Dense(100, name='Encoder-Output',activation='selu')(encoder_input_divider3)

model = Model(inputs=[vector,position],outputs=[output_vector,output_vector2])
#custom_adam=Adam(lr=0.0005)
model.compile(
    optimizer='adam',
    loss='mean_squared_error',
)

Using TensorFlow backend.
D:\anaconda3\envs\dec2\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\anaconda3\envs\dec2\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\anaconda3\envs\dec2\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\anaconda3\envs\dec2\lib\site-packages\tensorflow\python\framework\dtypes.py:519: Fu




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



In [2]:
from bpemb import BPEmb
from collections import Iterable
import numpy as np

bpemb_ru = BPEmb(lang="ru", dim=100, vs=3000)

def textreader(st):
    texts = []
    with open("D:/112/decoder/Data/data3000/np_rev2.txt", "r", encoding='utf-8') as fr:
        i=0
        for line in fr:
            i+=1
            if i<st:      
                continue
            texts.append(line.replace('\n',''))
            if i==st+499:
                break
    return texts

def word_vectors_creator(data):
    l=len(data)
    kol= np.zeros(shape=(l))
    pieces=[]
    for i in range(l):
        s = bpemb_ru.encode_ids(data[i])
        kol[i]+=len(s)
        pieces.append(s)
    kol = kol.astype(np.int)
    out = np.zeros(shape=(np.sum(kol),100))
    #out2 = np.zeros(shape=(np.sum(kol),100))
    buf = 0
    for g in range(l):        
        for k in range(0,kol[g]):
            out[buf]+=bpemb_ru.emb.vectors[pieces[g][k]]
            '''
            for m in range(kol[g]):
                if m!=k:
                    out2[buf]+=(bpemb_ru.emb.vectors[pieces[g][m]]/kol[g])
            buf+=1
            '''
    return out, kol       #, out2

def vectorsextender(data,kol):
    out = np.zeros(shape=(np.sum(kol),512))
    out2 = np.zeros(shape=(np.sum(kol)))
    buf=0
    for g in range(data.shape[0]):
        for k in range(kol[g]):
            out[buf]+=data[g]
            #out2[buf]+=np.sum(data[g])/512
            buf+=1
    return out#, out2

def positioncreator(kol):
    out = np.zeros(shape=(np.sum(kol)))
    buf=0
    for g in range(len(kol)):
        out[buf:buf+kol[g]]+=np.arange(1,kol[g]+1)
    return out
    

In [3]:
from keras.models import load_model
model = load_model ('D:/112/decoder/models/mse_sen_to_word0_100.h5')

Using TensorFlow backend.
D:\anaconda3\envs\dec2\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\anaconda3\envs\dec2\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\anaconda3\envs\dec2\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\anaconda3\envs\dec2\lib\site-packages\tensorflow\python\framework\dtypes.py:519: Fu



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




In [4]:
from sklearn.utils import shuffle

start=1
n=501
e=502
for i in range(101,200):
    vectors_train = np.load('D:/112/decoder/Data/data3000/news_vectors'+str(999-i)+'.npy')
    #print(start)
    texts = textreader(start)
    start+=500
    #print(len(texts))

    for j in range(5):
        tt, kol  = word_vectors_creator(texts[0+j*100:(1+j)*100])
        vtt = vectorsextender(vectors_train[0+j*100:(1+j)*100], kol)
        pos = positioncreator(kol)
        tt, vtt,pos = shuffle(tt, vtt,pos, random_state=0)

        if n%20==0:
            model.fit(
                x=[vtt,
                   pos],
                y = [vtt,
                    tt],
                epochs=e, initial_epoch=n,
                validation_split=0.05,
                verbose=1,
                batch_size=4
            )
            if n%500==0:
                model.save('D:/112/decoder/models/mse_sen_to_word'+str(d)+'_'+str(i)+'.h5')
                print('\n')
        else:
            model.fit(
                x=[vtt,
                   pos],
                y = [vtt,
                    tt],
                epochs=e, initial_epoch=n,
                validation_split=0.05,
                verbose=0,
                batch_size=4
            )
        n+=1
        e+=1

Train on 4578 samples, validate on 241 samples
Epoch 521/521
4578/4578 [==============================] - 17s 4ms/step - loss: 7.8072 - Encoder-Garbedge_loss: 0.0132 - Encoder-Output_loss: 7.7940 - val_loss: 1.9301 - val_Encoder-Garbedge_loss: 0.0138 - val_Encoder-Output_loss: 1.9163
Train on 5409 samples, validate on 285 samples
Epoch 541/541
5409/5409 [==============================] - 21s 4ms/step - loss: 7.8029 - Encoder-Garbedge_loss: 0.0131 - Encoder-Output_loss: 7.7898 - val_loss: 1.8076 - val_Encoder-Garbedge_loss: 0.0128 - val_Encoder-Output_loss: 1.7947
Train on 3873 samples, validate on 204 samples
Epoch 561/561
3873/3873 [==============================] - 14s 4ms/step - loss: 6.8133 - Encoder-Garbedge_loss: 0.0130 - Encoder-Output_loss: 6.8003 - val_loss: 1.7637 - val_Encoder-Garbedge_loss: 0.0120 - val_Encoder-Output_loss: 1.7517
Train on 6058 samples, validate on 319 samples
Epoch 581/581
6058/6058 [==============================] - 22s 4ms/step - loss: 8.3074 - Encoder-G

In [5]:
from sklearn.utils import shuffle


#n=0
#e=1
for d in range(1,20):
    start=1
    print('Epoch: '+str(d)+'\n'+"**********************************\n")
    for i in range(200):
        vectors_train = np.load('D:/112/decoder/Data/data3000/news_vectors'+str(999-i)+'.npy')
        #print(start)
        texts = textreader(start)
        start+=500
        #print(len(texts))

        for j in range(5):
            tt, kol  = word_vectors_creator(texts[0+j*100:(1+j)*100])
            vtt = vectorsextender(vectors_train[0+j*100:(1+j)*100], kol)
            pos = positioncreator(kol)
            tt, vtt,pos = shuffle(tt, vtt,pos, random_state=0)

            if n%20==0:
                model.fit(
                    x=[vtt,
                       pos],
                    y = [vtt,
                        tt],
                    epochs=e, initial_epoch=n,
                    validation_split=0.05,
                    verbose=1,
                    batch_size=4
                )
                if n%500==0:
                    model.save('D:/112/decoder/models/mse_sen_to_word'+str(d)+'_'+str(i)+'.h5')
                    print('\n')
            else:
                model.fit(
                    x=[vtt,
                       pos],
                    y = [vtt,
                        tt],
                    epochs=e, initial_epoch=n,
                    validation_split=0.05,
                    verbose=0,
                    batch_size=4
                )
            n+=1
            e+=1
    model.save('D:/112/decoder/models/mse_sen_to_word'+str(d)+'.h5')
    print('\n\n\n')

Epoch: 1
**********************************

Train on 4323 samples, validate on 228 samples
Epoch 1001/1001
4323/4323 [==============================] - 18s 4ms/step - loss: 6.7881 - Encoder-Garbedge_loss: 0.0132 - Encoder-Output_loss: 6.7749 - val_loss: 1.7102 - val_Encoder-Garbedge_loss: 0.0102 - val_Encoder-Output_loss: 1.7000


Train on 2870 samples, validate on 152 samples
Epoch 1021/1021
2870/2870 [==============================] - 12s 4ms/step - loss: 10.4405 - Encoder-Garbedge_loss: 0.0130 - Encoder-Output_loss: 10.4275 - val_loss: 1.8683 - val_Encoder-Garbedge_loss: 0.0138 - val_Encoder-Output_loss: 1.8545
Train on 5088 samples, validate on 268 samples
Epoch 1041/1041
5088/5088 [==============================] - 20s 4ms/step - loss: 7.5186 - Encoder-Garbedge_loss: 0.0134 - Encoder-Output_loss: 7.5051 - val_loss: 1.7735 - val_Encoder-Garbedge_loss: 0.0117 - val_Encoder-Output_loss: 1.7617
Train on 4287 samples, validate on 226 samples
Epoch 1061/1061
4287/4287 [================

3729/3729 [==============================] - 15s 4ms/step - loss: 6.6796 - Encoder-Garbedge_loss: 0.0133 - Encoder-Output_loss: 6.6663 - val_loss: 1.9029 - val_Encoder-Garbedge_loss: 0.0176 - val_Encoder-Output_loss: 1.8853
Train on 4362 samples, validate on 230 samples
Epoch 1581/1581
4362/4362 [==============================] - 17s 4ms/step - loss: 7.6802 - Encoder-Garbedge_loss: 0.0135 - Encoder-Output_loss: 7.6667 - val_loss: 1.8991 - val_Encoder-Garbedge_loss: 0.0137 - val_Encoder-Output_loss: 1.8855
Train on 3657 samples, validate on 193 samples
Epoch 1601/1601
3657/3657 [==============================] - 14s 4ms/step - loss: 8.9883 - Encoder-Garbedge_loss: 0.0137 - Encoder-Output_loss: 8.9746 - val_loss: 1.7802 - val_Encoder-Garbedge_loss: 0.0184 - val_Encoder-Output_loss: 1.7618
Train on 4241 samples, validate on 224 samples
Epoch 1621/1621
4241/4241 [==============================] - 17s 4ms/step - loss: 6.8969 - Encoder-Garbedge_loss: 0.0135 - Encoder-Output_loss: 6.8834 - va

5628/5628 [==============================] - 22s 4ms/step - loss: 8.0676 - Encoder-Garbedge_loss: 0.0135 - Encoder-Output_loss: 8.0542 - val_loss: 1.6816 - val_Encoder-Garbedge_loss: 0.0125 - val_Encoder-Output_loss: 1.6691
Train on 4883 samples, validate on 257 samples
Epoch 2141/2141
4883/4883 [==============================] - 19s 4ms/step - loss: 7.7098 - Encoder-Garbedge_loss: 0.0133 - Encoder-Output_loss: 7.6964 - val_loss: 1.7436 - val_Encoder-Garbedge_loss: 0.0127 - val_Encoder-Output_loss: 1.7309
Train on 4391 samples, validate on 232 samples
Epoch 2161/2161
4391/4391 [==============================] - 17s 4ms/step - loss: 5.7956 - Encoder-Garbedge_loss: 0.0133 - Encoder-Output_loss: 5.7823 - val_loss: 1.7096 - val_Encoder-Garbedge_loss: 0.0097 - val_Encoder-Output_loss: 1.6999
Train on 5474 samples, validate on 289 samples
Epoch 2181/2181
5474/5474 [==============================] - 21s 4ms/step - loss: 7.1513 - Encoder-Garbedge_loss: 0.0133 - Encoder-Output_loss: 7.1380 - va

Train on 4006 samples, validate on 211 samples
Epoch 2701/2701
4006/4006 [==============================] - 15s 4ms/step - loss: 9.1494 - Encoder-Garbedge_loss: 0.0133 - Encoder-Output_loss: 9.1362 - val_loss: 1.7141 - val_Encoder-Garbedge_loss: 0.0141 - val_Encoder-Output_loss: 1.7000
Train on 4707 samples, validate on 248 samples
Epoch 2721/2721
4707/4707 [==============================] - 17s 4ms/step - loss: 6.5053 - Encoder-Garbedge_loss: 0.0134 - Encoder-Output_loss: 6.4919 - val_loss: 2.2086 - val_Encoder-Garbedge_loss: 0.0112 - val_Encoder-Output_loss: 2.1974
Train on 2228 samples, validate on 118 samples
Epoch 2741/2741
2228/2228 [==============================] - 9s 4ms/step - loss: 9.4222 - Encoder-Garbedge_loss: 0.0141 - Encoder-Output_loss: 9.4080 - val_loss: 1.7130 - val_Encoder-Garbedge_loss: 0.0131 - val_Encoder-Output_loss: 1.6999
Train on 4584 samples, validate on 242 samples
Epoch 2761/2761
4584/4584 [==============================] - 17s 4ms/step - loss: 6.5923 - En

4440/4440 [==============================] - 16s 4ms/step - loss: 6.3916 - Encoder-Garbedge_loss: 0.0135 - Encoder-Output_loss: 6.3780 - val_loss: 1.7135 - val_Encoder-Garbedge_loss: 0.0134 - val_Encoder-Output_loss: 1.7000
Train on 4445 samples, validate on 234 samples
Epoch 3281/3281
4445/4445 [==============================] - 16s 4ms/step - loss: 7.5736 - Encoder-Garbedge_loss: 0.0134 - Encoder-Output_loss: 7.5602 - val_loss: 1.8990 - val_Encoder-Garbedge_loss: 0.0137 - val_Encoder-Output_loss: 1.8853
Train on 4634 samples, validate on 244 samples
Epoch 3301/3301
4634/4634 [==============================] - 17s 4ms/step - loss: 7.1929 - Encoder-Garbedge_loss: 0.0132 - Encoder-Output_loss: 7.1797 - val_loss: 1.6807 - val_Encoder-Garbedge_loss: 0.0114 - val_Encoder-Output_loss: 1.6693
Train on 3845 samples, validate on 203 samples
Epoch 3321/3321
3845/3845 [==============================] - 14s 4ms/step - loss: 6.8273 - Encoder-Garbedge_loss: 0.0135 - Encoder-Output_loss: 6.8138 - va

...

Train on 4964 samples, validate on 262 samples
Epoch 8961/8961
4964/4964 [==============================] - 18s 4ms/step - loss: 5.6659 - Encoder-Garbedge_loss: 0.0133 - Encoder-Output_loss: 5.6526 - val_loss: 1.9865 - val_Encoder-Garbedge_loss: 0.0086 - val_Encoder-Output_loss: 1.9779
Train on 5469 samples, validate on 288 samples
Epoch 8981/8981
5469/5469 [==============================] - 20s 4ms/step - loss: 7.0128 - Encoder-Garbedge_loss: 0.0132 - Encoder-Output_loss: 6.9996 - val_loss: 1.7861 - val_Encoder-Garbedge_loss: 0.0146 - val_Encoder-Output_loss: 1.7715der-Garbedge_loss: 0.0132 - Encoder-Outp




Epoch: 9
**********************************

Train on 4323 samples, validate on 228 samples
Epoch 9001/9001
4323/4323 [==============================] - 16s 4ms/step - loss: 6.6326 - Encoder-Garbedge_loss: 0.0133 - Encoder-Output_loss: 6.6194 - val_loss: 1.6797 - val_Encoder-Garbedge_loss: 0.0106 - val_Encoder-Output_loss: 1.6691


Train on 2870 samples, validate on 152 samples
E

3935/3935 [==============================] - 14s 4ms/step - loss: 6.9245 - Encoder-Garbedge_loss: 0.0131 - Encoder-Output_loss: 6.9113 - val_loss: 1.8660 - val_Encoder-Garbedge_loss: 0.0115 - val_Encoder-Output_loss: 1.8545
Train on 4031 samples, validate on 213 samples
Epoch 9541/9541
4031/4031 [==============================] - 14s 4ms/step - loss: 7.1608 - Encoder-Garbedge_loss: 0.0131 - Encoder-Output_loss: 7.1477 - val_loss: 1.7112 - val_Encoder-Garbedge_loss: 0.0112 - val_Encoder-Output_loss: 1.7000
Train on 3729 samples, validate on 197 samples
Epoch 9561/9561
3729/3729 [==============================] - 13s 4ms/step - loss: 6.6064 - Encoder-Garbedge_loss: 0.0133 - Encoder-Output_loss: 6.5931 - val_loss: 1.7724 - val_Encoder-Garbedge_loss: 0.0106 - val_Encoder-Output_loss: 1.7618
Train on 4362 samples, validate on 230 samples
Epoch 9581/9581
4362/4362 [==============================] - 15s 4ms/step - loss: 7.5594 - Encoder-Garbedge_loss: 0.0134 - Encoder-Output_loss: 7.5460 - va

3027/3027 [==============================] - 11s 4ms/step - loss: 8.1489 - Encoder-Garbedge_loss: 0.0134 - Encoder-Output_loss: 8.1355 - val_loss: 1.8059 - val_Encoder-Garbedge_loss: 0.0127 - val_Encoder-Output_loss: 1.7932
Train on 4067 samples, validate on 215 samples
Epoch 10101/10101
4067/4067 [==============================] - 14s 4ms/step - loss: 7.1745 - Encoder-Garbedge_loss: 0.0133 - Encoder-Output_loss: 7.1613 - val_loss: 1.6528 - val_Encoder-Garbedge_loss: 0.0146 - val_Encoder-Output_loss: 1.6382
Train on 5628 samples, validate on 297 samples
Epoch 10121/10121
5628/5628 [==============================] - 20s 4ms/step - loss: 8.0830 - Encoder-Garbedge_loss: 0.0135 - Encoder-Output_loss: 8.0695 - val_loss: 1.7673 - val_Encoder-Garbedge_loss: 0.0134 - val_Encoder-Output_loss: 1.7539
Train on 4883 samples, validate on 257 samples
Epoch 10141/10141
4883/4883 [==============================] - 18s 4ms/step - loss: 7.6179 - Encoder-Garbedge_loss: 0.0135 - Encoder-Output_loss: 7.604

2373/2373 [==============================] - 8s 4ms/step - loss: 5.4181 - Encoder-Garbedge_loss: 0.0133 - Encoder-Output_loss: 5.4048 - val_loss: 1.6856 - val_Encoder-Garbedge_loss: 0.0166 - val_Encoder-Output_loss: 1.6690
Train on 4390 samples, validate on 232 samples
Epoch 10661/10661
4390/4390 [==============================] - 16s 4ms/step - loss: 7.5414 - Encoder-Garbedge_loss: 0.0134 - Encoder-Output_loss: 7.5280 - val_loss: 1.6493 - val_Encoder-Garbedge_loss: 0.0111 - val_Encoder-Output_loss: 1.6382
Train on 4099 samples, validate on 216 samples
Epoch 10681/10681
4099/4099 [==============================] - 15s 4ms/step - loss: 7.6620 - Encoder-Garbedge_loss: 0.0134 - Encoder-Output_loss: 7.6486 - val_loss: 1.8033 - val_Encoder-Garbedge_loss: 0.0103 - val_Encoder-Output_loss: 1.7931
Train on 4006 samples, validate on 211 samples
Epoch 10701/10701
4006/4006 [==============================] - 14s 4ms/step - loss: 9.1917 - Encoder-Garbedge_loss: 0.0136 - Encoder-Output_loss: 9.1781

KeyboardInterrupt: 

In [6]:
model.save('D:/decoder/models/sen_to_word'+str(i)+'.h5')